In [0]:
from tensorflow import __version__

In [3]:
__version__

'2.0.0'

In [0]:
import pandas as pd

In [0]:
oo=pd.read_csv('sampled_reviews.csv')

In [6]:
oo.head()

,Unnamed: 0,Unnamed: 0.1,cleaned_summary,cleaned_text
0,178885,178892,good sweetener,is very good for all it does not contain any s...
1,240275,240285,smiley likes it,while our dog smiley will eat just about anyth...
2,298013,298024,high on low on flavor,if you want to eat a lot for only a few calori...
3,538510,538537,death rain this name says it,the first time i tried these was from the vend...
4,200181,200189,post selects banana nut crunch cereal review,great grains banana nut crunch cereal is reall...


In [0]:
oo=oo.drop(labels=['Unnamed: 0','Unnamed: 0.1'],axis=1)

In [8]:
oo.sample(4)

,cleaned_summary,cleaned_text
13658,dilution required mixing required,the directions on the bottle state that one sh...
1861,quick morning brew,been drinking one or two cups of senseo medium...
7637,earl grey still a winner,the flavor is good but not as robust as it wou...
1109,not,acai in my book is over we no longer drink the...


In [9]:
'''
MY UNDERSTANDING OF ENCODER-DECODER MODELS

For text sumnmarization we're going to use a sequence to sequence encoder-decoder architecture
as it is ideal for solving problems where the input and output are of different lengths.

An encoder decoder architecture has two phases:

1) Training Phase: The encoder lstm reads the entire input sequence where at each timestep we are 
                   feeding one word at a time into the encoder.
                   The encoder will capture the context from these input sequences.
                   The so called "context" would be the last hidden state and cell state produced by the encoder
                   after capturing context from the input sequence.

                   The decoder takes a target sequence and encoder hidden and cell states as input and predict the 
                   target sequence incremented by one time step.
                   
                   The decoder can take the context of "Oh my God I hate this coffee!" from the encoder
                   and the word "hate" and predict the word "coffee". That's what the decoder is being trained for.

                   A <start> token has to be added to a decoder's input sequence so as to be able to be able to predict
                   the first word of the decoder output sequence using the combination of the <start> token and the context
                   coming in from the encoder. The decoder can take the context from "Oh my god, I hate this cofee" and the
                   <start> token and predict the first word of the target which would be a summary in our case which would be
                   "hate".

                   The <stop> token has to be added to the target sequence so that a decoder can predict the end of a sentence
                   given the context from an encoder and the last word from the target sequence. So given the context from 
                   "Oh my God, I hate this coffee" and "coffee" and predict the <stop> token.

2) Inference Phase: Once we have our encoder trained to convert input sequence and convert it to a context and a decoder
                    that can take that context and some word from a target sequence and predict the next word we use the model
                    to start generating target sequence.

                    1)Encode entire input sequence and intialize decoder with the states returned from the encoder
                    2)Pass the <start> as input into the decoder.
                    3)Run the decoder for one time step. The output will be the probabilities for the next word
                      and the word with the highest probability will be selected.
                    4)Next, this high probability selected word will be passed into the decoder, which will update the 
                      the states of the decoder which will predict the next word
                    5)This process of passing the word again and again by passing it into the decoder and updating states at each output
                      will continue till either a <stop> token is predicted or the length of the predicted sequence that you've been 
                      accumulating is equal to the maximum length of any target sequence in the training dataset.
'''
print('this is here to list my assumptions and to debug any errors I get during programming the model.')

this is here to list my assumptions and to debug any errors I get during programming the model.


In [0]:
oo['target_summary']=oo.cleaned_summary.apply(lambda text: "<START> "+str(text)+" <STOP>")

In [84]:
oo.head(1)

,cleaned_summary,cleaned_text,target_summary
0,good sweetener,is very good for all it does not contain any s...,<START> good sweetener <STOP>


In [0]:
oo=oo[oo.cleaned_text.str.len()<= 500]

In [16]:
oo.sample(3)

,cleaned_summary,cleaned_text,target_summary
5706,these are,i keep them in my drawer at work for snacking ...,<START> these are <STOP>
10547,very cherry,just what i not too not to as the ones in the ...,<START> very cherry <STOP>
2662,uneven production quality,i have purchased a carton containing least hal...,<START> uneven production quality <STOP>


In [17]:
oo.shape

(11533, 3)

In [0]:
x_maxlen=max([len(s) for s in oo.cleaned_text])
y_maxlen=max([len(s) for s in oo.target_summary])

In [19]:
x_maxlen

500

In [0]:
unique_words=set()

for seq in oo.cleaned_text:
  for word in seq.split(' '):
    if word not in unique_words:
      unique_words.add(word)
for seq in oo.target_summary:
  for word in seq.split(' '):
    if word not in unique_words:
      unique_words.add(word)

In [0]:
unique_words=list(unique_words)

In [0]:
word_index={value:index for index,value in enumerate(unique_words)}

In [0]:
reverse_word_index={value:key for key,value in word_index.items()}

In [24]:
oo.shape

(11533, 3)

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_tokenizer=Tokenizer()
x_tokenizer.fit_on_texts(oo.cleaned_text)
X=x_tokenizer.texts_to_sequences(oo.cleaned_text)
X=pad_sequences(X,maxlen=x_maxlen,padding='post')

In [0]:
y_tokenizer=Tokenizer()
y_tokenizer.fit_on_texts(oo.target_summary)
y=y_tokenizer.texts_to_sequences(oo.target_summary)
y=pad_sequences(y,maxlen=y_maxlen,padding='post')

In [28]:
X.shape

(11533, 500)

In [29]:
y.shape

(11533, 117)

In [0]:
x_vocab_size=len(x_tokenizer.word_index)+1

In [0]:
y_vocab_size=len(y_tokenizer.word_index)+1

In [0]:
from tensorflow.keras.models import Model

In [0]:
from tensorflow.keras.layers import Input, Embedding, Dense, LSTM

In [0]:
latent_dim=256
encoder_input=Input(shape=(x_maxlen,),name='encoder_input')
enc_embedding_layer=Embedding(x_vocab_size,latent_dim,trainable=True,name='encoder_embedding')
enc_embedding=enc_embedding_layer(encoder_input)

encoder_lstm1=LSTM(latent_dim,return_state=True,name='encoder_lstm')
encoder_output1,state_h,state_c=encoder_lstm1(enc_embedding)

encoder_states=[state_h,state_c]

decoder_input=Input(shape=(None,),name='decoder_input')# try changing to None
decoder_embedding_layer=Embedding(y_vocab_size,latent_dim,trainable=True,name='decoder_embedding')
decoder_embedding=decoder_embedding_layer(decoder_input)

decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True,name='decoder_lstm')
decoder_output, _, _=decoder_lstm(decoder_embedding,initial_state=encoder_states)

decoder_dense=Dense(y_vocab_size,activation='softmax')
decoder_output=decoder_dense(decoder_output)

model=Model([encoder_input,decoder_input],decoder_output)

In [50]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 500)]        0                                            
__________________________________________________________________________________________________
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
encoder_embedding (Embedding)   (None, 500, 256)     3477248     encoder_input[0][0]              
__________________________________________________________________________________________________
decoder_embedding (Embedding)   (None, None, 256)    1126912     decoder_input[0][0]              
____________________________________________________________________________________________

In [0]:
model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [41]:
X.shape

(11533, 500)

In [43]:
y.shape

(11533, 117)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [57]:
history=model.fit([X_train,y_train[:,:-1]], y_train.reshape(y_train.shape[0],y_train.shape[1], 1)[:,1:] ,epochs=10,validation_split=0.1)

Train on 7784 samples, validate on 865 samples
Epoch 1/10
7784/7784 [==============================] - 1122s 144ms/sample - loss: 0.1903 - accuracy: 0.9729 - val_loss: 0.1891 - val_accuracy: 0.9724
Epoch 2/10
7784/7784 [==============================] - 1130s 145ms/sample - loss: 0.1847 - accuracy: 0.9729 - val_loss: 0.1866 - val_accuracy: 0.9727
Epoch 3/10
7784/7784 [==============================] - 1135s 146ms/sample - loss: 0.1810 - accuracy: 0.9731 - val_loss: 0.1855 - val_accuracy: 0.9729
Epoch 4/10
7784/7784 [==============================] - 1150s 148ms/sample - loss: 0.1771 - accuracy: 0.9733 - val_loss: 0.1817 - val_accuracy: 0.9731
Epoch 5/10
7784/7784 [==============================] - 1140s 146ms/sample - loss: 0.1730 - accuracy: 0.9735 - val_loss: 0.1801 - val_accuracy: 0.9734
Epoch 6/10
7784/7784 [==============================] - 1132s 145ms/sample - loss: 0.1692 - accuracy: 0.9738 - val_loss: 0.1790 - val_accuracy: 0.9734
Epoch 7/10
7784/7784 [=========================

In [0]:
model.save('summarizer.h5')

In [0]:
decoder_input_data=y_train[:,:-1]

In [60]:
y_train.shape

(8649, 117)

In [88]:
decoder_input_data.shape

(8649, 116)

In [90]:
decoder_target_data.shape

(8649, 116, 1)

In [91]:
X_train.shape

(8649, 500)

In [92]:
X_train[0]

array([   2,   52,   73,  153,    4,  207,   51,  289,  472,   41,  191,
        375,  319,    2,   97,    8,    2,  187,    1,  123, 1018,  600,
          5,  600,    3,  174,    4,   38, 1662,    9,    1,  880,  285,
          8,   48,    7,   67, 2271,   11,  472,   54,   49,    1,   51,
        289,   78,  274,    2,   70,    4,  198,  364,    3,   42,  460,
          5,   62,    8,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [93]:
decoder_input_data[0]

array([ 2, 22,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

In [95]:
decoder_target_data[0]

array([[22],
       [ 1],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 0],

In [0]:
#Inference Phase
encoder_model=Model(inputs=encoder_input,outputs=[encoder_output1,state_h,state_c])
#this model uses the trained encoder layer to encode complete input sequences

decoder_state_input_h=Input(shape=(latent_dim,))
decoder_state_input_c=Input(shape=(latent_dim,))#shape should match the number of units in the decoder LSTM cell.
#decoder_hidden_state_input=Input(shape=(x_maxlen,latent_dim))

decoder_embedding_2=decoder_embedding_layer(decoder_input)
decoder_states_inputs=[decoder_state_input_h,decoder_state_input_c]

decoder_output_2,state_h2,state_c2=decoder_lstm(decoder_embedding_2,initial_state=decoder_states_inputs)

decoder_states=[state_h2,state_c2]

decoder_output_2=decoder_dense(decoder_output_2)

decoder_model=Model([decoder_input]+decoder_states_inputs,
                    [decoder_output_2]+decoder_states)

In [0]:
target_word_index=y_tokenizer.word_index
input_word_index=x_tokenizer.word_index

reverse_target_word_index={value:key for key,value in target_word_index.items()}
reverse_input_word_index={value:key for key,value in input_word_index.items()}

In [99]:
target_word_index['start']

2

In [0]:
def decode_sequence(input_seq):
  encoder_out, encoder_h, encoder_c= encoder_model.predict(input_seq)

  target_seq=np.zeros((1,1))

  target_seq[0,0]=target_word_index['start']

  stop_condition=False
  decoded_sentence=''

  while not stop_condition:
    output_tokens, h, c= decoder_model.predict([target_seq]+[encoder_out,encoder_h,encoder_c])

    sampled_token_index=np.argmax(output_tokens[0, -1,:])
    sampled_token=reverse_target_word_index[sampled_token_index]

    if sampled_token != 'stop':
      decoded_sentence+=' '+sampled_token
    else:
      stop_condition=True

    if len(decoded_sentence) >= (y_maxlen-1):
        stop_condition=True
    
    target_seq=np.zeros((1,1))
    target_seq[0,0] = sampled_token_index

    encoder_h, encoder_c = h, c
  
  return decoded_sentence

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))

    # Chose the 'start' word as the first word of the target sequence
    target_seq[0, 0] = target_word_index['start']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]

        if(sampled_token!='stop'):
            decoded_sentence += ' '+sampled_token

            # Exit condition: either hit max length or find stop word.
            if (sampled_token == 'stop' or len(decoded_sentence.split()) >= (y_maxlen-1)):
                stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [0]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
      if((i!=0 and i!=target_word_index['start']) and i!=target_word_index['stop']):
        newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
      if(i!=0):
        newString=newString+reverse_input_word_index[i]+' '
    return newString

In [111]:
import numpy as np
for i in range(len(X_train)):
  print("Review:",seq2text(X_test[i]))
  print("Original summary:",seq2summary(y_test[i]))
  print("Predicted summary:",decode_sequence(X_test[i].reshape(1,x_maxlen)))
  print("\n")

Review: my dog has developed bad allergies later when he was years naturally tried several premium dog food and feeding special formula foods are the only kind i found that my skin he was on the fish formula for over a but after a while it started to trigger his lately i feed him with the potato duck so far works really his fur is perfectly healthy and 
Original summary: good quality food for special needs dog 
Predicted summary:  nan nasty expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expected expect

KeyboardInterrupt: ignored

In [0]:
target_